<a href="https://colab.research.google.com/github/IT24102088/Plant-Disease-Detection-System-PG-NO-147/blob/main/combined_preprocessing_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Combined Preprocessing Pipeline**

In [2]:
from IPython.display import clear_output
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = 'supuntheekshanam'
os.environ['KAGGLE_KEY'] = '97fb6c99381c9d6779d3f904ef279dce'
clear_output()

In [3]:
!kaggle datasets download -d "vipoooool/new-plant-diseases-dataset" -p /content/drive/MyDrive/datasets/

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
 99% 2.68G/2.70G [00:41<00:00, 154MB/s]
100% 2.70G/2.70G [00:41<00:00, 69.5MB/s]


In [4]:
!unzip /content/drive/MyDrive/datasets/new-plant-diseases-dataset.zip
clear_output()

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

**Load training and validation data and resize the images**

In [6]:

training_set = tf.keras.utils.image_dataset_from_directory(
    '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/',
    image_size=(224, 224),
    batch_size=32,
    validation_split=None,
    subset=None,
)

training_set = tf.keras.utils.image_dataset_from_directory(
    '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/',
    image_size=(224, 224),
    batch_size=32,
    validation_split=None,
    subset=None,
)



Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.


**Rescaling**

In [7]:
from tensorflow.keras.layers import Rescaling
training_set = training_set.map(lambda x, y: (Rescaling(1./255)(x), y))

**Geometric Augmentation**

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
geometric_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomTranslation(0.1, 0.1),
])

def apply_geometric_augmentation(image, label):
    image = geometric_augmentation(image, training=True)
    return image, label

training_set = training_set.map(apply_geometric_augmentation)


**Color_Augmentation**

In [9]:
color_augmentation = tf.keras.Sequential([
    layers.RandomBrightness(0.2),
    layers.RandomContrast(0.2),
    layers.RandomSaturation(0.2),
    layers.RandomHue(0.1),
])

def apply_color_augmentation(image, label):
    image = color_augmentation(image, training=True)
    return image, label

training_set = training_set.map(apply_color_augmentation)

**Gaussian Noise**

In [10]:
from tensorflow.keras.layers import GaussianNoise

def add_gaussian_noise(image, stddev=0.05):
    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=stddev)
    return tf.clip_by_value(image + noise, 0.0, 1.0)

def apply_noise_augmentation(image, label):
    image = add_gaussian_noise(image)
    return image,label

training_set = training_set.map(apply_noise_augmentation)



**Histogram Equalization**

In [11]:
def histogram_equalization_rgb(image):

    # Convert to YUV color space
    image_yuv = tf.image.rgb_to_yuv(image)

    # Apply histogram equalization to Y channel (luminance)
    y_channel = image_yuv[..., 0:1]

    # Enhanced contrast adjustment (simulating histogram equalization)
    y_eq = tf.image.adjust_contrast(y_channel, 2.0)
    y_eq = tf.clip_by_value(y_eq, 0.0, 1.0)

    # Combine back with original UV channels
    image_yuv_eq = tf.concat([y_eq, image_yuv[..., 1:2], image_yuv[..., 2:3]], axis=-1)

    # Convert back to RGB
    return tf.image.yuv_to_rgb(image_yuv_eq)

def apply_histogram_equalization(image, label):
    image = histogram_equalization_rgb(image)
    return image, label

training_set = training_set.map(apply_histogram_equalization)

